In [1]:
import pandas as pd
import numpy as np
from vars import currency_features
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200 
pd.options.display.max_colwidth = 100

###  Define clipping strategy

In [4]:
data = pd.read_csv('../data/processed/data.csv')
data = data.sort_values(['symbol', 'calendarYear'])

In [5]:
top5 = np.arange(0.95, 0.99, 0.01)
low5 = np.arange(0.01, 0.06, 0.01)
data.describe(percentiles=np.concatenate((low5,top5),axis=None)).T

,count,mean,std,min,1%,2%,3%,4%,5%,50%,95%,96%,97%,98%,99%,max
calendarYear,12948.00000,2008.56487,7.51566,1996.00000,1996.00000,1996.00000,1996.00000,1997.00000,1997.00000,2009.00000,2020.00000,2020.00000,2021.00000,2021.00000,2021.00000,2021.00000
cashAndCashEquivalents,10017.00000,4.49559,22.01475,0.00000,0.00400,0.01149,0.02192,0.03204,0.04159,0.89155,13.14370,16.32495,20.40962,34.29834,71.98697,744.66456
shortTermInvestments,10017.00000,5.95378,33.38426,-0.29200,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,21.24719,32.32719,50.09868,69.89009,126.88299,472.58670
cashAndShortTermInvestments,10017.00000,10.44937,49.20020,-0.01757,0.00518,0.01448,0.02639,0.03742,0.04803,1.20833,38.44157,49.89589,72.36092,96.53403,193.86586,1036.42236
netReceivables,10017.00000,5.11402,19.83944,-0.07260,0.00000,0.00000,0.00000,0.00000,0.00269,1.32569,18.46170,21.59246,28.21069,37.62153,66.97531,537.31134
inventory,10158.00000,1.70067,4.53188,-3.15138,0.00000,0.00000,0.00000,0.00000,0.00000,0.47149,7.47780,8.80850,10.01118,12.05021,15.57559,99.83866
otherCurrentAssets,10017.00000,11.33974,123.02541,-3839.05048,-37.62844,-8.76741,-0.56692,0.00000,0.00000,0.38448,29.24909,46.05322,68.17342,110.06975,387.88142,1549.28554
totalCurrentAssets,10017.00000,26.56416,155.35463,-3835.77783,0.00000,0.00000,0.00000,0.01691,0.23559,4.27293,83.24721,103.55668,128.58293,198.88785,533.67899,2688.80825
propertyPlantEquipmentNet,10017.00000,9.73909,26.45109,-0.35396,0.00000,0.00000,0.00000,0.00000,0.00000,2.37119,36.94380,40.95785,49.68009,65.97726,108.50646,508.32641
goodwill,10017.00000,4.52295,10.64822,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.90679,20.48355,25.11110,30.24708,37.43882,54.65059,159.18262


# Exploration

In [6]:
def get_context(data, symbol, columns = data.columns.tolist()):
    idx_mask = data.symbol == symbol
    return data.loc[idx_mask, columns]

In [7]:
context=[
    'symbol', 
    'calendarYear', 
    'target', 
    'revenue',
    'totalAssets', 
    'netIncome', 
    'totalEquity', 
    'finalLink'
    ]

In [8]:
get_context(data, 'CHK', context)

,symbol,calendarYear,target,revenue,totalAssets,netIncome,totalEquity,finalLink


# Maybe check increases

In [ ]:
also_compute = [
    'weightedAverageShsOut',
    'weightedAverageShsOutDil',
    'preferredStock',
    'commonStock',
]
for col in currency_features + also_compute:
    yoy_colname = col + '_yoy'
    data[yoy_colname] = data[col].pct_change(1)